To practice writing performant Julia code I am attempting to improve the speed of part 1. As a reminder, the terminal output after the last run was:

        Processing folder: info
        Finished in 4.35 seconds
        Processing folder: local-monthly-gross-returns
        Finished in 130.5 seconds
        Processing folder: local-monthly-net-returns
        Finished in 148.06 seconds
        Processing folder: monthly-costs
        Finished in 63.1 seconds
        Processing folder: monthly-morningstar-category
        Finished in 467.8 seconds
        Processing folder: monthly-net-assets
        Finished in 237.36 seconds
        Processing folder: usd-monthly-gross-returns
        Finished in 85.24 seconds
        Processing folder: usd-monthly-net-returns
        Finished in 100.15 seconds

The total runtime of the script (missing the time taken to load the info dataframe initially), then is:

In [6]:
println(round((4.35+130.5+148.06+63.1+467.8+237.36+85.24+100.15)/60, digits=2), " minutes")

20.61 minutes


The first change made is to employ parallel processing through @threads. The new terminal output is:

I lost the terminal output after the first run because I had to stop it partway through, but had recorded two of the times manually. It looked something like:

        Regrouping files...
        Processed folder info in 12.6 seconds
        Processed folder monthly-costs in 1412.8 seconds
        Processed folder monthly-usd-gross-returns in 1730.4 seconds

In [7]:
println("monthly-costs took $(round(1412.8/60, digits=2)) minutes")
println("monthly-usd-gross-returns took $(round(1730.4/60, digits=2)) minutes")


monthly-costs took 23.55 minutes
monthly-usd-gross-returns took 28.84 minutes


The ratio of time for usd-gross-returns to time for monthly-costs remained roughly the same, but took 22x as long, so there was no benefit to parallelism at all and the overtime time taken was substantially increased.

I need to see the form of the monthly-morningstar-category data but can't get a preview to open in VS Code

In [2]:
using CSV
using DataFrames

data = CSV.read("../Data/Raw Data/Mutual Funds/monthly-morningstar-category/mf_monthly-morningstar-category_part-1.csv", DataFrame,
                limit=50, missingstring="")

Row,Name,FundId,SecId,Morningstar Category \n1990-01,Morningstar Category \n1990-02,Morningstar Category \n1990-03,Morningstar Category \n1990-04,Morningstar Category \n1990-05,Morningstar Category \n1990-06,Morningstar Category \n1990-07,Morningstar Category \n1990-08,Morningstar Category \n1990-09,Morningstar Category \n1990-10,Morningstar Category \n1990-11,Morningstar Category \n1990-12,Morningstar Category \n1991-01,Morningstar Category \n1991-02,Morningstar Category \n1991-03,Morningstar Category \n1991-04,Morningstar Category \n1991-05,Morningstar Category \n1991-06,Morningstar Category \n1991-07,Morningstar Category \n1991-08,Morningstar Category \n1991-09,Morningstar Category \n1991-10,Morningstar Category \n1991-11,Morningstar Category \n1991-12,Morningstar Category \n1992-01,Morningstar Category \n1992-02,Morningstar Category \n1992-03,Morningstar Category \n1992-04,Morningstar Category \n1992-05,Morningstar Category \n1992-06,Morningstar Category \n1992-07,Morningstar Category \n1992-08,Morningstar Category \n1992-09,Morningstar Category \n1992-10,Morningstar Category \n1992-11,Morningstar Category \n1992-12,Morningstar Category \n1993-01,Morningstar Category \n1993-02,Morningstar Category \n1993-03,Morningstar Category \n1993-04,Morningstar Category \n1993-05,Morningstar Category \n1993-06,Morningstar Category \n1993-07,Morningstar Category \n1993-08,Morningstar Category \n1993-09,Morningstar Category \n1993-10,Morningstar Category \n1993-11,Morningstar Category \n1993-12,Morningstar Category \n1994-01,Morningstar Category \n1994-02,Morningstar Category \n1994-03,Morningstar Category \n1994-04,Morningstar Category \n1994-05,Morningstar Category \n1994-06,Morningstar Category \n1994-07,Morningstar Category \n1994-08,Morningstar Category \n1994-09,Morningstar Category \n1994-10,Morningstar Category \n1994-11,Morningstar Category \n1994-12,Morningstar Category \n1995-01,Morningstar Category \n1995-02,Morningstar Category \n1995-03,Morningstar Category \n1995-04,Morningstar Category \n1995-05,Morningstar Category \n1995-06,Morningstar Category \n1995-07,Morningstar Category \n1995-08,Morningstar Category \n1995-09,Morningstar Category \n1995-10,Morningstar Category \n1995-11,Morningstar Category \n1995-12,Morningstar Category \n1996-01,Morningstar Category \n1996-02,Morningstar Category \n1996-03,Morningstar Category \n1996-04,Morningstar Category \n1996-05,Morningstar Category \n1996-06,Morningstar Category \n1996-07,Morningstar Category \n1996-08,Morningstar Category \n1996-09,Morningstar Category \n1996-10,Morningstar Category \n1996-11,Morningstar Category \n1996-12,Morningstar Category \n1997-01,Morningstar Category \n1997-02,Morningstar Category \n1997-03,Morningstar Category \n1997-04,Morningstar Category \n1997-05,Morningstar Category \n1997-06,Morningstar Category \n1997-07,Morningstar Category \n1997-08,Morningstar Category \n1997-09,Morningstar Category \n1997-10,Morningstar Category \n1997-11,Morningstar Category \n1997-12,Morningstar Category \n1998-01,⋯
,String,String15,String15,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,⋯
1,@rgentum Canadian Equity Portfolio,FS000091C6,F00000NE43,,,,,,,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,miss

In [19]:
ismissing(data[1, "Morningstar Category \n1990-01"])

false

That's very strange. Why are missing values being read as empty strings?

In [20]:
data2 = CSV.read("../Data/Raw Data/Mutual Funds/monthly-costs/mf_monthly-costs_part-1.csv", DataFrame,
                limit=10)

Row,Name,FundId,SecId,Representative Cost Ex Transaction Fee (Unannualised) \n1990-01,Representative Cost Ex Transaction Fee (Unannualised) \n1990-02,Representative Cost Ex Transaction Fee (Unannualised) \n1990-03,Representative Cost Ex Transaction Fee (Unannualised) \n1990-04,Representative Cost Ex Transaction Fee (Unannualised) \n1990-05,Representative Cost Ex Transaction Fee (Unannualised) \n1990-06,Representative Cost Ex Transaction Fee (Unannualised) \n1990-07,Representative Cost Ex Transaction Fee (Unannualised) \n1990-08,Representative Cost Ex Transaction Fee (Unannualised) \n1990-09,Representative Cost Ex Transaction Fee (Unannualised) \n1990-10,Representative Cost Ex Transaction Fee (Unannualised) \n1990-11,Representative Cost Ex Transaction Fee (Unannualised) \n1990-12,Representative Cost Ex Transaction Fee (Unannualised) \n1991-01,Representative Cost Ex Transaction Fee (Unannualised) \n1991-02,Representative Cost Ex Transaction Fee (Unannualised) \n1991-03,Representative Cost Ex Transaction Fee (Unannualised) \n1991-04,Representative Cost Ex Transaction Fee (Unannualised) \n1991-05,Representative Cost Ex Transaction Fee (Unannualised) \n1991-06,Representative Cost Ex Transaction Fee (Unannualised) \n1991-07,Representative Cost Ex Transaction Fee (Unannualised) \n1991-08,Representative Cost Ex Transaction Fee (Unannualised) \n1991-09,Representative Cost Ex Transaction Fee (Unannualised) \n1991-10,Representative Cost Ex Transaction Fee (Unannualised) \n1991-11,Representative Cost Ex Transaction Fee (Unannualised) \n1991-12,Representative Cost Ex Transaction Fee (Unannualised) \n1992-01,Representative Cost Ex Transaction Fee (Unannualised) \n1992-02,Representative Cost Ex Transaction Fee (Unannualised) \n1992-03,Representative Cost Ex Transaction Fee (Unannualised) \n1992-04,Representative Cost Ex Transaction Fee (Unannualised) \n1992-05,Representative Cost Ex Transaction Fee (Unannualised) \n1992-06,Representative Cost Ex Transaction Fee (Unannualised) \n1992-07,Representative Cost Ex Transaction Fee (Unannualised) \n1992-08,Representative Cost Ex Transaction Fee (Unannualised) \n1992-09,Representative Cost Ex Transaction Fee (Unannualised) \n1992-10,Representative Cost Ex Transaction Fee (Unannualised) \n1992-11,Representative Cost Ex Transaction Fee (Unannualised) \n1992-12,Representative Cost Ex Transaction Fee (Unannualised) \n1993-01,Representative Cost Ex Transaction Fee (Unannualised) \n1993-02,Representative Cost Ex Transaction Fee (Unannualised) \n1993-03,Representative Cost Ex Transaction Fee (Unannualised) \n1993-04,Representative Cost Ex Transaction Fee (Unannualised) \n1993-05,Representative Cost Ex Transaction Fee (Unannualised) \n1993-06,Representative Cost Ex Transaction Fee (Unannualised) \n1993-07,Representative Cost Ex Transaction Fee (Unannualised) \n1993-08,Representative Cost Ex Transaction Fee (Unannualised) \n1993-09,Representative Cost Ex Transaction Fee (Unannualised) \n1993-10,Representative Cost Ex Transaction Fee (Unannualised) \n1993-11,Representative Cost Ex Transaction Fee (Unannualised) \n1993-12,Representative Cost Ex Transaction Fee (Unannualised) \n1994-01,Representative Cost Ex Transaction Fee (Unannualised) \n1994-02,Representative Cost Ex Transaction Fee (Unannualised) \n1994-03,Representative Cost Ex Transaction Fee (Unannualised) \n1994-04,Representative Cost Ex Transaction Fee (Unannualised) \n1994-05,Representative Cost Ex Transaction Fee (Unannualised) \n1994-06,Representative Cost Ex Transaction Fee (Unannualised) \n1994-07,Representative Cost Ex Transaction Fee (Unannualised) \n1994-08,Representative Cost Ex Transaction Fee (Unannualised) \n1994-09,Representative Cost Ex Transaction Fee (Unannualised) \n1994-10,Representative Cost Ex Transaction Fee (Unannualised) \n1994-11,Representative Cost Ex Transaction Fee (Unannualised) \n1994-12,Representative Cost Ex Transaction Fee (Unannualised) \n1995-01,Representative Cost Ex Transaction Fee (Unannualised) \n1995-02,Representa

This csv also uses empty strings, but those strings are correctly read as missing.

In [27]:
# Reread category data
data_new = CSV.read("../Data/Raw Data/Mutual Funds/monthly-morningstar-category/mf_monthly-morningstar-category_part-1.csv",
                    DataFrame, types=Union{Missing,String}, limit=10)

Row,Name,FundId,SecId,Morningstar Category \n1990-01,Morningstar Category \n1990-02,Morningstar Category \n1990-03,Morningstar Category \n1990-04,Morningstar Category \n1990-05,Morningstar Category \n1990-06,Morningstar Category \n1990-07,Morningstar Category \n1990-08,Morningstar Category \n1990-09,Morningstar Category \n1990-10,Morningstar Category \n1990-11,Morningstar Category \n1990-12,Morningstar Category \n1991-01,Morningstar Category \n1991-02,Morningstar Category \n1991-03,Morningstar Category \n1991-04,Morningstar Category \n1991-05,Morningstar Category \n1991-06,Morningstar Category \n1991-07,Morningstar Category \n1991-08,Morningstar Category \n1991-09,Morningstar Category \n1991-10,Morningstar Category \n1991-11,Morningstar Category \n1991-12,Morningstar Category \n1992-01,Morningstar Category \n1992-02,Morningstar Category \n1992-03,Morningstar Category \n1992-04,Morningstar Category \n1992-05,Morningstar Category \n1992-06,Morningstar Category \n1992-07,Morningstar Category \n1992-08,Morningstar Category \n1992-09,Morningstar Category \n1992-10,Morningstar Category \n1992-11,Morningstar Category \n1992-12,Morningstar Category \n1993-01,Morningstar Category \n1993-02,Morningstar Category \n1993-03,Morningstar Category \n1993-04,Morningstar Category \n1993-05,Morningstar Category \n1993-06,Morningstar Category \n1993-07,Morningstar Category \n1993-08,Morningstar Category \n1993-09,Morningstar Category \n1993-10,Morningstar Category \n1993-11,Morningstar Category \n1993-12,Morningstar Category \n1994-01,Morningstar Category \n1994-02,Morningstar Category \n1994-03,Morningstar Category \n1994-04,Morningstar Category \n1994-05,Morningstar Category \n1994-06,Morningstar Category \n1994-07,Morningstar Category \n1994-08,Morningstar Category \n1994-09,Morningstar Category \n1994-10,Morningstar Category \n1994-11,Morningstar Category \n1994-12,Morningstar Category \n1995-01,Morningstar Category \n1995-02,Morningstar Category \n1995-03,Morningstar Category \n1995-04,Morningstar Category \n1995-05,Morningstar Category \n1995-06,Morningstar Category \n1995-07,Morningstar Category \n1995-08,Morningstar Category \n1995-09,Morningstar Category \n1995-10,Morningstar Category \n1995-11,Morningstar Category \n1995-12,Morningstar Category \n1996-01,Morningstar Category \n1996-02,Morningstar Category \n1996-03,Morningstar Category \n1996-04,Morningstar Category \n1996-05,Morningstar Category \n1996-06,Morningstar Category \n1996-07,Morningstar Category \n1996-08,Morningstar Category \n1996-09,Morningstar Category \n1996-10,Morningstar Category \n1996-11,Morningstar Category \n1996-12,Morningstar Category \n1997-01,Morningstar Category \n1997-02,Morningstar Category \n1997-03,Morningstar Category \n1997-04,Morningstar Category \n1997-05,Morningstar Category \n1997-06,Morningstar Category \n1997-07,Morningstar Category \n1997-08,Morningstar Category \n1997-09,Morningstar Category \n1997-10,Morningstar Category \n1997-11,Morningstar Category \n1997-12,Morningstar Category \n1998-01,⋯
,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,⋯
1,@rgentum Canadian Equity Portfolio,FS000091C6,F00000NE43,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,⋯
2,@rgent

In [33]:
# What about
data3 = CSV.read("../Data/Raw Data/Mutual Funds/monthly-morningstar-category/mf_monthly-morningstar-category_part-1.csv",
                    DataFrame, limit=10)

Row,Name,FundId,SecId,Morningstar Category \n1990-01,Morningstar Category \n1990-02,Morningstar Category \n1990-03,Morningstar Category \n1990-04,Morningstar Category \n1990-05,Morningstar Category \n1990-06,Morningstar Category \n1990-07,Morningstar Category \n1990-08,Morningstar Category \n1990-09,Morningstar Category \n1990-10,Morningstar Category \n1990-11,Morningstar Category \n1990-12,Morningstar Category \n1991-01,Morningstar Category \n1991-02,Morningstar Category \n1991-03,Morningstar Category \n1991-04,Morningstar Category \n1991-05,Morningstar Category \n1991-06,Morningstar Category \n1991-07,Morningstar Category \n1991-08,Morningstar Category \n1991-09,Morningstar Category \n1991-10,Morningstar Category \n1991-11,Morningstar Category \n1991-12,Morningstar Category \n1992-01,Morningstar Category \n1992-02,Morningstar Category \n1992-03,Morningstar Category \n1992-04,Morningstar Category \n1992-05,Morningstar Category \n1992-06,Morningstar Category \n1992-07,Morningstar Category \n1992-08,Morningstar Category \n1992-09,Morningstar Category \n1992-10,Morningstar Category \n1992-11,Morningstar Category \n1992-12,Morningstar Category \n1993-01,Morningstar Category \n1993-02,Morningstar Category \n1993-03,Morningstar Category \n1993-04,Morningstar Category \n1993-05,Morningstar Category \n1993-06,Morningstar Category \n1993-07,Morningstar Category \n1993-08,Morningstar Category \n1993-09,Morningstar Category \n1993-10,Morningstar Category \n1993-11,Morningstar Category \n1993-12,Morningstar Category \n1994-01,Morningstar Category \n1994-02,Morningstar Category \n1994-03,Morningstar Category \n1994-04,Morningstar Category \n1994-05,Morningstar Category \n1994-06,Morningstar Category \n1994-07,Morningstar Category \n1994-08,Morningstar Category \n1994-09,Morningstar Category \n1994-10,Morningstar Category \n1994-11,Morningstar Category \n1994-12,Morningstar Category \n1995-01,Morningstar Category \n1995-02,Morningstar Category \n1995-03,Morningstar Category \n1995-04,Morningstar Category \n1995-05,Morningstar Category \n1995-06,Morningstar Category \n1995-07,Morningstar Category \n1995-08,Morningstar Category \n1995-09,Morningstar Category \n1995-10,Morningstar Category \n1995-11,Morningstar Category \n1995-12,Morningstar Category \n1996-01,Morningstar Category \n1996-02,Morningstar Category \n1996-03,Morningstar Category \n1996-04,Morningstar Category \n1996-05,Morningstar Category \n1996-06,Morningstar Category \n1996-07,Morningstar Category \n1996-08,Morningstar Category \n1996-09,Morningstar Category \n1996-10,Morningstar Category \n1996-11,Morningstar Category \n1996-12,Morningstar Category \n1997-01,Morningstar Category \n1997-02,Morningstar Category \n1997-03,Morningstar Category \n1997-04,Morningstar Category \n1997-05,Morningstar Category \n1997-06,Morningstar Category \n1997-07,Morningstar Category \n1997-08,Morningstar Category \n1997-09,Morningstar Category \n1997-10,Morningstar Category \n1997-11,Morningstar Category \n1997-12,Morningstar Category \n1998-01,⋯
,String,String15,String15,Missing,Missing,Missing,Missing,Missing,Missing,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,⋯
1,@rgentum Canadian Equity Portfolio,FS000091C6,F00000NE43,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,mi

In [34]:
ismissing(data3[1, "Morningstar Category \n1990-01"])

true

I had added in quoted=false, but then changed back to quoted=true and deleted it again. Then the call worked as usual...

In [35]:
data4 = CSV.read("../Data/Raw Data/Mutual Funds/monthly-morningstar-category/mf_monthly-morningstar-category_part-1.csv",
                    DataFrame, limit=10)

Row,Name,FundId,SecId,Morningstar Category \n1990-01,Morningstar Category \n1990-02,Morningstar Category \n1990-03,Morningstar Category \n1990-04,Morningstar Category \n1990-05,Morningstar Category \n1990-06,Morningstar Category \n1990-07,Morningstar Category \n1990-08,Morningstar Category \n1990-09,Morningstar Category \n1990-10,Morningstar Category \n1990-11,Morningstar Category \n1990-12,Morningstar Category \n1991-01,Morningstar Category \n1991-02,Morningstar Category \n1991-03,Morningstar Category \n1991-04,Morningstar Category \n1991-05,Morningstar Category \n1991-06,Morningstar Category \n1991-07,Morningstar Category \n1991-08,Morningstar Category \n1991-09,Morningstar Category \n1991-10,Morningstar Category \n1991-11,Morningstar Category \n1991-12,Morningstar Category \n1992-01,Morningstar Category \n1992-02,Morningstar Category \n1992-03,Morningstar Category \n1992-04,Morningstar Category \n1992-05,Morningstar Category \n1992-06,Morningstar Category \n1992-07,Morningstar Category \n1992-08,Morningstar Category \n1992-09,Morningstar Category \n1992-10,Morningstar Category \n1992-11,Morningstar Category \n1992-12,Morningstar Category \n1993-01,Morningstar Category \n1993-02,Morningstar Category \n1993-03,Morningstar Category \n1993-04,Morningstar Category \n1993-05,Morningstar Category \n1993-06,Morningstar Category \n1993-07,Morningstar Category \n1993-08,Morningstar Category \n1993-09,Morningstar Category \n1993-10,Morningstar Category \n1993-11,Morningstar Category \n1993-12,Morningstar Category \n1994-01,Morningstar Category \n1994-02,Morningstar Category \n1994-03,Morningstar Category \n1994-04,Morningstar Category \n1994-05,Morningstar Category \n1994-06,Morningstar Category \n1994-07,Morningstar Category \n1994-08,Morningstar Category \n1994-09,Morningstar Category \n1994-10,Morningstar Category \n1994-11,Morningstar Category \n1994-12,Morningstar Category \n1995-01,Morningstar Category \n1995-02,Morningstar Category \n1995-03,Morningstar Category \n1995-04,Morningstar Category \n1995-05,Morningstar Category \n1995-06,Morningstar Category \n1995-07,Morningstar Category \n1995-08,Morningstar Category \n1995-09,Morningstar Category \n1995-10,Morningstar Category \n1995-11,Morningstar Category \n1995-12,Morningstar Category \n1996-01,Morningstar Category \n1996-02,Morningstar Category \n1996-03,Morningstar Category \n1996-04,Morningstar Category \n1996-05,Morningstar Category \n1996-06,Morningstar Category \n1996-07,Morningstar Category \n1996-08,Morningstar Category \n1996-09,Morningstar Category \n1996-10,Morningstar Category \n1996-11,Morningstar Category \n1996-12,Morningstar Category \n1997-01,Morningstar Category \n1997-02,Morningstar Category \n1997-03,Morningstar Category \n1997-04,Morningstar Category \n1997-05,Morningstar Category \n1997-06,Morningstar Category \n1997-07,Morningstar Category \n1997-08,Morningstar Category \n1997-09,Morningstar Category \n1997-10,Morningstar Category \n1997-11,Morningstar Category \n1997-12,Morningstar Category \n1998-01,⋯
,String,String15,String15,Missing,Missing,Missing,Missing,Missing,Missing,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,⋯
1,@rgentum Canadian Equity Portfolio,FS000091C6,F00000NE43,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,mi

In [36]:
# Here is an exact copy of the offending code from above
using CSV
using DataFrames

data = CSV.read("../Data/Raw Data/Mutual Funds/monthly-morningstar-category/mf_monthly-morningstar-category_part-1.csv", DataFrame,
                limit=50, missingstring="")

Row,Name,FundId,SecId,Morningstar Category \n1990-01,Morningstar Category \n1990-02,Morningstar Category \n1990-03,Morningstar Category \n1990-04,Morningstar Category \n1990-05,Morningstar Category \n1990-06,Morningstar Category \n1990-07,Morningstar Category \n1990-08,Morningstar Category \n1990-09,Morningstar Category \n1990-10,Morningstar Category \n1990-11,Morningstar Category \n1990-12,Morningstar Category \n1991-01,Morningstar Category \n1991-02,Morningstar Category \n1991-03,Morningstar Category \n1991-04,Morningstar Category \n1991-05,Morningstar Category \n1991-06,Morningstar Category \n1991-07,Morningstar Category \n1991-08,Morningstar Category \n1991-09,Morningstar Category \n1991-10,Morningstar Category \n1991-11,Morningstar Category \n1991-12,Morningstar Category \n1992-01,Morningstar Category \n1992-02,Morningstar Category \n1992-03,Morningstar Category \n1992-04,Morningstar Category \n1992-05,Morningstar Category \n1992-06,Morningstar Category \n1992-07,Morningstar Category \n1992-08,Morningstar Category \n1992-09,Morningstar Category \n1992-10,Morningstar Category \n1992-11,Morningstar Category \n1992-12,Morningstar Category \n1993-01,Morningstar Category \n1993-02,Morningstar Category \n1993-03,Morningstar Category \n1993-04,Morningstar Category \n1993-05,Morningstar Category \n1993-06,Morningstar Category \n1993-07,Morningstar Category \n1993-08,Morningstar Category \n1993-09,Morningstar Category \n1993-10,Morningstar Category \n1993-11,Morningstar Category \n1993-12,Morningstar Category \n1994-01,Morningstar Category \n1994-02,Morningstar Category \n1994-03,Morningstar Category \n1994-04,Morningstar Category \n1994-05,Morningstar Category \n1994-06,Morningstar Category \n1994-07,Morningstar Category \n1994-08,Morningstar Category \n1994-09,Morningstar Category \n1994-10,Morningstar Category \n1994-11,Morningstar Category \n1994-12,Morningstar Category \n1995-01,Morningstar Category \n1995-02,Morningstar Category \n1995-03,Morningstar Category \n1995-04,Morningstar Category \n1995-05,Morningstar Category \n1995-06,Morningstar Category \n1995-07,Morningstar Category \n1995-08,Morningstar Category \n1995-09,Morningstar Category \n1995-10,Morningstar Category \n1995-11,Morningstar Category \n1995-12,Morningstar Category \n1996-01,Morningstar Category \n1996-02,Morningstar Category \n1996-03,Morningstar Category \n1996-04,Morningstar Category \n1996-05,Morningstar Category \n1996-06,Morningstar Category \n1996-07,Morningstar Category \n1996-08,Morningstar Category \n1996-09,Morningstar Category \n1996-10,Morningstar Category \n1996-11,Morningstar Category \n1996-12,Morningstar Category \n1997-01,Morningstar Category \n1997-02,Morningstar Category \n1997-03,Morningstar Category \n1997-04,Morningstar Category \n1997-05,Morningstar Category \n1997-06,Morningstar Category \n1997-07,Morningstar Category \n1997-08,Morningstar Category \n1997-09,Morningstar Category \n1997-10,Morningstar Category \n1997-11,Morningstar Category \n1997-12,Morningstar Category \n1998-01,⋯
,String,String15,String15,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,⋯
1,@rgentum Canadian Equity Portfolio,FS000091C6,F00000NE43,,,,,,,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,miss

okay so it is still bugging, despite the other read working. What's the difference?

In [46]:
d = CSV.read("../Data/Raw Data/Mutual Funds/monthly-morningstar-category/mf_monthly-morningstar-category_part-1.csv", DataFrame,
                limit=13)

Row,Name,FundId,SecId,Morningstar Category \n1990-01,Morningstar Category \n1990-02,Morningstar Category \n1990-03,Morningstar Category \n1990-04,Morningstar Category \n1990-05,Morningstar Category \n1990-06,Morningstar Category \n1990-07,Morningstar Category \n1990-08,Morningstar Category \n1990-09,Morningstar Category \n1990-10,Morningstar Category \n1990-11,Morningstar Category \n1990-12,Morningstar Category \n1991-01,Morningstar Category \n1991-02,Morningstar Category \n1991-03,Morningstar Category \n1991-04,Morningstar Category \n1991-05,Morningstar Category \n1991-06,Morningstar Category \n1991-07,Morningstar Category \n1991-08,Morningstar Category \n1991-09,Morningstar Category \n1991-10,Morningstar Category \n1991-11,Morningstar Category \n1991-12,Morningstar Category \n1992-01,Morningstar Category \n1992-02,Morningstar Category \n1992-03,Morningstar Category \n1992-04,Morningstar Category \n1992-05,Morningstar Category \n1992-06,Morningstar Category \n1992-07,Morningstar Category \n1992-08,Morningstar Category \n1992-09,Morningstar Category \n1992-10,Morningstar Category \n1992-11,Morningstar Category \n1992-12,Morningstar Category \n1993-01,Morningstar Category \n1993-02,Morningstar Category \n1993-03,Morningstar Category \n1993-04,Morningstar Category \n1993-05,Morningstar Category \n1993-06,Morningstar Category \n1993-07,Morningstar Category \n1993-08,Morningstar Category \n1993-09,Morningstar Category \n1993-10,Morningstar Category \n1993-11,Morningstar Category \n1993-12,Morningstar Category \n1994-01,Morningstar Category \n1994-02,Morningstar Category \n1994-03,Morningstar Category \n1994-04,Morningstar Category \n1994-05,Morningstar Category \n1994-06,Morningstar Category \n1994-07,Morningstar Category \n1994-08,Morningstar Category \n1994-09,Morningstar Category \n1994-10,Morningstar Category \n1994-11,Morningstar Category \n1994-12,Morningstar Category \n1995-01,Morningstar Category \n1995-02,Morningstar Category \n1995-03,Morningstar Category \n1995-04,Morningstar Category \n1995-05,Morningstar Category \n1995-06,Morningstar Category \n1995-07,Morningstar Category \n1995-08,Morningstar Category \n1995-09,Morningstar Category \n1995-10,Morningstar Category \n1995-11,Morningstar Category \n1995-12,Morningstar Category \n1996-01,Morningstar Category \n1996-02,Morningstar Category \n1996-03,Morningstar Category \n1996-04,Morningstar Category \n1996-05,Morningstar Category \n1996-06,Morningstar Category \n1996-07,Morningstar Category \n1996-08,Morningstar Category \n1996-09,Morningstar Category \n1996-10,Morningstar Category \n1996-11,Morningstar Category \n1996-12,Morningstar Category \n1997-01,Morningstar Category \n1997-02,Morningstar Category \n1997-03,Morningstar Category \n1997-04,Morningstar Category \n1997-05,Morningstar Category \n1997-06,Morningstar Category \n1997-07,Morningstar Category \n1997-08,Morningstar Category \n1997-09,Morningstar Category \n1997-10,Morningstar Category \n1997-11,Morningstar Category \n1997-12,Morningstar Category \n1998-01,⋯
,String,String15,String15,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,⋯
1,@rgentum Canadian Equity Portfolio,FS000091C6,F00000NE43,,,,,,,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,miss

Loading 12 or fewer rows treats missing cells correctly, but the 13th row causes it to fail. Something about what's written in the 13th row?

In [47]:
# I will try editing the value in row 13 for the fourth column to a standard string, savings a temp file and reloading
d[13, "Morningstar Category \n1990-01"] = "temp"

CSV.write("temp.csv", d)

d2 = CSV.read("temp.csv", DataFrame)

Row,Name,FundId,SecId,Morningstar Category \n1990-01,Morningstar Category \n1990-02,Morningstar Category \n1990-03,Morningstar Category \n1990-04,Morningstar Category \n1990-05,Morningstar Category \n1990-06,Morningstar Category \n1990-07,Morningstar Category \n1990-08,Morningstar Category \n1990-09,Morningstar Category \n1990-10,Morningstar Category \n1990-11,Morningstar Category \n1990-12,Morningstar Category \n1991-01,Morningstar Category \n1991-02,Morningstar Category \n1991-03,Morningstar Category \n1991-04,Morningstar Category \n1991-05,Morningstar Category \n1991-06,Morningstar Category \n1991-07,Morningstar Category \n1991-08,Morningstar Category \n1991-09,Morningstar Category \n1991-10,Morningstar Category \n1991-11,Morningstar Category \n1991-12,Morningstar Category \n1992-01,Morningstar Category \n1992-02,Morningstar Category \n1992-03,Morningstar Category \n1992-04,Morningstar Category \n1992-05,Morningstar Category \n1992-06,Morningstar Category \n1992-07,Morningstar Category \n1992-08,Morningstar Category \n1992-09,Morningstar Category \n1992-10,Morningstar Category \n1992-11,Morningstar Category \n1992-12,Morningstar Category \n1993-01,Morningstar Category \n1993-02,Morningstar Category \n1993-03,Morningstar Category \n1993-04,Morningstar Category \n1993-05,Morningstar Category \n1993-06,Morningstar Category \n1993-07,Morningstar Category \n1993-08,Morningstar Category \n1993-09,Morningstar Category \n1993-10,Morningstar Category \n1993-11,Morningstar Category \n1993-12,Morningstar Category \n1994-01,Morningstar Category \n1994-02,Morningstar Category \n1994-03,Morningstar Category \n1994-04,Morningstar Category \n1994-05,Morningstar Category \n1994-06,Morningstar Category \n1994-07,Morningstar Category \n1994-08,Morningstar Category \n1994-09,Morningstar Category \n1994-10,Morningstar Category \n1994-11,Morningstar Category \n1994-12,Morningstar Category \n1995-01,Morningstar Category \n1995-02,Morningstar Category \n1995-03,Morningstar Category \n1995-04,Morningstar Category \n1995-05,Morningstar Category \n1995-06,Morningstar Category \n1995-07,Morningstar Category \n1995-08,Morningstar Category \n1995-09,Morningstar Category \n1995-10,Morningstar Category \n1995-11,Morningstar Category \n1995-12,Morningstar Category \n1996-01,Morningstar Category \n1996-02,Morningstar Category \n1996-03,Morningstar Category \n1996-04,Morningstar Category \n1996-05,Morningstar Category \n1996-06,Morningstar Category \n1996-07,Morningstar Category \n1996-08,Morningstar Category \n1996-09,Morningstar Category \n1996-10,Morningstar Category \n1996-11,Morningstar Category \n1996-12,Morningstar Category \n1997-01,Morningstar Category \n1997-02,Morningstar Category \n1997-03,Morningstar Category \n1997-04,Morningstar Category \n1997-05,Morningstar Category \n1997-06,Morningstar Category \n1997-07,Morningstar Category \n1997-08,Morningstar Category \n1997-09,Morningstar Category \n1997-10,Morningstar Category \n1997-11,Morningstar Category \n1997-12,Morningstar Category \n1998-01,⋯
,String,String15,String15,String31?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,⋯
1,@rgentum Canadian Equity Portfolio,FS000091C6,F00000NE43,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,

It fixed the whole table????

In [63]:
# Trying just a straight load, save temp, reload temp
d = CSV.read("../Data/Raw Data/Mutual Funds/monthly-morningstar-category/mf_monthly-morningstar-category_part-1.csv", DataFrame,
                limit=13)

CSV.write("temp2.csv", d)

d2 = CSV.read("temp2.csv", DataFrame)

CompositeException: TaskFailedException

    nested task error: BoundsError: attempt to access 45698-element Vector{UInt8} at index [45699]
    Stacktrace:
     [1] getindex
       @ .\array.jl:924 [inlined]
     [2] findchunkrowstart(ranges::Vector{Int64}, i::Int64, buf::Vector{UInt8}, opts::Parsers.Options, typemap::Dict{Type, Type}, downcast::Bool, ncols::Int64, rows_to_check::Int64, columns::Vector{CSV.Column}, origcoltypes::Vector{Type}, columnlock::ReentrantLock, stringtype::Any, totalbytes::Base.Threads.Atomic{Int64}, totalrows::Base.Threads.Atomic{Int64}, succeeded::Base.Threads.Atomic{Bool})
       @ CSV C:\Users\ozboy\.julia\packages\CSV\b8ebJ\src\detection.jl:436
     [3] macro expansion
       @ C:\Users\ozboy\.julia\packages\CSV\b8ebJ\src\detection.jl:470 [inlined]
     [4] (::CSV.var"#16#17"{Vector{UInt8}, Parsers.Options, Vector{Int64}, Int64, Vector{CSV.Column}, DataType, Dict{Type, Type}, Bool, Int64, Vector{Type}, ReentrantLock, Base.Threads.Atomic{Bool}, Base.Threads

I'm having a lot of trouble trying to load these files.

In [64]:
d = CSV.read("../Data/Raw Data/Mutual Funds/monthly-morningstar-category/mf_monthly-morningstar-category_part-1.csv", DataFrame,
            limit=13)

Row,Name,FundId,SecId,Morningstar Category \n1990-01,Morningstar Category \n1990-02,Morningstar Category \n1990-03,Morningstar Category \n1990-04,Morningstar Category \n1990-05,Morningstar Category \n1990-06,Morningstar Category \n1990-07,Morningstar Category \n1990-08,Morningstar Category \n1990-09,Morningstar Category \n1990-10,Morningstar Category \n1990-11,Morningstar Category \n1990-12,Morningstar Category \n1991-01,Morningstar Category \n1991-02,Morningstar Category \n1991-03,Morningstar Category \n1991-04,Morningstar Category \n1991-05,Morningstar Category \n1991-06,Morningstar Category \n1991-07,Morningstar Category \n1991-08,Morningstar Category \n1991-09,Morningstar Category \n1991-10,Morningstar Category \n1991-11,Morningstar Category \n1991-12,Morningstar Category \n1992-01,Morningstar Category \n1992-02,Morningstar Category \n1992-03,Morningstar Category \n1992-04,Morningstar Category \n1992-05,Morningstar Category \n1992-06,Morningstar Category \n1992-07,Morningstar Category \n1992-08,Morningstar Category \n1992-09,Morningstar Category \n1992-10,Morningstar Category \n1992-11,Morningstar Category \n1992-12,Morningstar Category \n1993-01,Morningstar Category \n1993-02,Morningstar Category \n1993-03,Morningstar Category \n1993-04,Morningstar Category \n1993-05,Morningstar Category \n1993-06,Morningstar Category \n1993-07,Morningstar Category \n1993-08,Morningstar Category \n1993-09,Morningstar Category \n1993-10,Morningstar Category \n1993-11,Morningstar Category \n1993-12,Morningstar Category \n1994-01,Morningstar Category \n1994-02,Morningstar Category \n1994-03,Morningstar Category \n1994-04,Morningstar Category \n1994-05,Morningstar Category \n1994-06,Morningstar Category \n1994-07,Morningstar Category \n1994-08,Morningstar Category \n1994-09,Morningstar Category \n1994-10,Morningstar Category \n1994-11,Morningstar Category \n1994-12,Morningstar Category \n1995-01,Morningstar Category \n1995-02,Morningstar Category \n1995-03,Morningstar Category \n1995-04,Morningstar Category \n1995-05,Morningstar Category \n1995-06,Morningstar Category \n1995-07,Morningstar Category \n1995-08,Morningstar Category \n1995-09,Morningstar Category \n1995-10,Morningstar Category \n1995-11,Morningstar Category \n1995-12,Morningstar Category \n1996-01,Morningstar Category \n1996-02,Morningstar Category \n1996-03,Morningstar Category \n1996-04,Morningstar Category \n1996-05,Morningstar Category \n1996-06,Morningstar Category \n1996-07,Morningstar Category \n1996-08,Morningstar Category \n1996-09,Morningstar Category \n1996-10,Morningstar Category \n1996-11,Morningstar Category \n1996-12,Morningstar Category \n1997-01,Morningstar Category \n1997-02,Morningstar Category \n1997-03,Morningstar Category \n1997-04,Morningstar Category \n1997-05,Morningstar Category \n1997-06,Morningstar Category \n1997-07,Morningstar Category \n1997-08,Morningstar Category \n1997-09,Morningstar Category \n1997-10,Morningstar Category \n1997-11,Morningstar Category \n1997-12,Morningstar Category \n1998-01,⋯
,String,String15,String15,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,⋯
1,@rgentum Canadian Equity Portfolio,FS000091C6,F00000NE43,,,,,,,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,miss

In [69]:
# Okay I manually copies the file because when resaving the quotes were being lost

d = CSV.read("copy_df.csv", DataFrame, limit=13)

Row,Name,FundId,SecId,Morningstar Category \n1990-01,Morningstar Category \n1990-02,Morningstar Category \n1990-03,Morningstar Category \n1990-04,Morningstar Category \n1990-05,Morningstar Category \n1990-06,Morningstar Category \n1990-07,Morningstar Category \n1990-08,Morningstar Category \n1990-09,Morningstar Category \n1990-10,Morningstar Category \n1990-11,Morningstar Category \n1990-12,Morningstar Category \n1991-01,Morningstar Category \n1991-02,Morningstar Category \n1991-03,Morningstar Category \n1991-04,Morningstar Category \n1991-05,Morningstar Category \n1991-06,Morningstar Category \n1991-07,Morningstar Category \n1991-08,Morningstar Category \n1991-09,Morningstar Category \n1991-10,Morningstar Category \n1991-11,Morningstar Category \n1991-12,Morningstar Category \n1992-01,Morningstar Category \n1992-02,Morningstar Category \n1992-03,Morningstar Category \n1992-04,Morningstar Category \n1992-05,Morningstar Category \n1992-06,Morningstar Category \n1992-07,Morningstar Category \n1992-08,Morningstar Category \n1992-09,Morningstar Category \n1992-10,Morningstar Category \n1992-11,Morningstar Category \n1992-12,Morningstar Category \n1993-01,Morningstar Category \n1993-02,Morningstar Category \n1993-03,Morningstar Category \n1993-04,Morningstar Category \n1993-05,Morningstar Category \n1993-06,Morningstar Category \n1993-07,Morningstar Category \n1993-08,Morningstar Category \n1993-09,Morningstar Category \n1993-10,Morningstar Category \n1993-11,Morningstar Category \n1993-12,Morningstar Category \n1994-01,Morningstar Category \n1994-02,Morningstar Category \n1994-03,Morningstar Category \n1994-04,Morningstar Category \n1994-05,Morningstar Category \n1994-06,Morningstar Category \n1994-07,Morningstar Category \n1994-08,Morningstar Category \n1994-09,Morningstar Category \n1994-10,Morningstar Category \n1994-11,Morningstar Category \n1994-12,Morningstar Category \n1995-01,Morningstar Category \n1995-02,Morningstar Category \n1995-03,Morningstar Category \n1995-04,Morningstar Category \n1995-05,Morningstar Category \n1995-06,Morningstar Category \n1995-07,Morningstar Category \n1995-08,Morningstar Category \n1995-09,Morningstar Category \n1995-10,Morningstar Category \n1995-11,Morningstar Category \n1995-12,Morningstar Category \n1996-01,Morningstar Category \n1996-02,Morningstar Category \n1996-03,Morningstar Category \n1996-04,Morningstar Category \n1996-05,Morningstar Category \n1996-06,Morningstar Category \n1996-07,Morningstar Category \n1996-08,Morningstar Category \n1996-09,Morningstar Category \n1996-10,Morningstar Category \n1996-11,Morningstar Category \n1996-12,Morningstar Category \n1997-01,Morningstar Category \n1997-02,Morningstar Category \n1997-03,Morningstar Category \n1997-04,Morningstar Category \n1997-05,Morningstar Category \n1997-06,Morningstar Category \n1997-07,Morningstar Category \n1997-08,Morningstar Category \n1997-09,Morningstar Category \n1997-10,Morningstar Category \n1997-11,Morningstar Category \n1997-12,Morningstar Category \n1998-01,⋯
,String,String15,String15,String31?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,⋯
1,@rgentum Canadian Equity Portfolio,FS000091C6,F00000NE43,missing,,,,,,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,mis

Aha. There is something about "EAA Fund EUR Money Market - Short Term" that trips up the missing values for the rest of the column in columns 4-9, but even then not in the remainder of the columns?? It's so confusing.

In [70]:
d = CSV.read("copy_df.csv", DataFrame, types=String63)

Row,Name,FundId,SecId,Morningstar Category \n1990-01,Morningstar Category \n1990-02,Morningstar Category \n1990-03,Morningstar Category \n1990-04,Morningstar Category \n1990-05,Morningstar Category \n1990-06,Morningstar Category \n1990-07,Morningstar Category \n1990-08,Morningstar Category \n1990-09,Morningstar Category \n1990-10,Morningstar Category \n1990-11,Morningstar Category \n1990-12,Morningstar Category \n1991-01,Morningstar Category \n1991-02,Morningstar Category \n1991-03,Morningstar Category \n1991-04,Morningstar Category \n1991-05,Morningstar Category \n1991-06,Morningstar Category \n1991-07,Morningstar Category \n1991-08,Morningstar Category \n1991-09,Morningstar Category \n1991-10,Morningstar Category \n1991-11,Morningstar Category \n1991-12,Morningstar Category \n1992-01,Morningstar Category \n1992-02,Morningstar Category \n1992-03,Morningstar Category \n1992-04,Morningstar Category \n1992-05,Morningstar Category \n1992-06,Morningstar Category \n1992-07,Morningstar Category \n1992-08,Morningstar Category \n1992-09,Morningstar Category \n1992-10,Morningstar Category \n1992-11,Morningstar Category \n1992-12,Morningstar Category \n1993-01,Morningstar Category \n1993-02,Morningstar Category \n1993-03,Morningstar Category \n1993-04,Morningstar Category \n1993-05,Morningstar Category \n1993-06,Morningstar Category \n1993-07,Morningstar Category \n1993-08,Morningstar Category \n1993-09,Morningstar Category \n1993-10,Morningstar Category \n1993-11,Morningstar Category \n1993-12,Morningstar Category \n1994-01,Morningstar Category \n1994-02,Morningstar Category \n1994-03,Morningstar Category \n1994-04,Morningstar Category \n1994-05,Morningstar Category \n1994-06,Morningstar Category \n1994-07,Morningstar Category \n1994-08,Morningstar Category \n1994-09,Morningstar Category \n1994-10,Morningstar Category \n1994-11,Morningstar Category \n1994-12,Morningstar Category \n1995-01,Morningstar Category \n1995-02,Morningstar Category \n1995-03,Morningstar Category \n1995-04,Morningstar Category \n1995-05,Morningstar Category \n1995-06,Morningstar Category \n1995-07,Morningstar Category \n1995-08,Morningstar Category \n1995-09,Morningstar Category \n1995-10,Morningstar Category \n1995-11,Morningstar Category \n1995-12,Morningstar Category \n1996-01,Morningstar Category \n1996-02,Morningstar Category \n1996-03,Morningstar Category \n1996-04,Morningstar Category \n1996-05,Morningstar Category \n1996-06,Morningstar Category \n1996-07,Morningstar Category \n1996-08,Morningstar Category \n1996-09,Morningstar Category \n1996-10,Morningstar Category \n1996-11,Morningstar Category \n1996-12,Morningstar Category \n1997-01,Morningstar Category \n1997-02,Morningstar Category \n1997-03,Morningstar Category \n1997-04,Morningstar Category \n1997-05,Morningstar Category \n1997-06,Morningstar Category \n1997-07,Morningstar Category \n1997-08,Morningstar Category \n1997-09,Morningstar Category \n1997-10,Morningstar Category \n1997-11,Morningstar Category \n1997-12,Morningstar Category \n1998-01,⋯
,String63,String63,String63,String63?,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,String63,⋯
1,@rgentum Canadian Equity Portfolio,FS000091C6,F00000NE43,missin

It's clear that CSV.read just isn't parsing correctly. So now I'm not sure what to do.

I'm going to test some code to delete all "" in all file parts, and if it works then I'm going to run it here on all of the files.

In [3]:
# Read the copied data

d = CSV.read("copy_df.csv", DataFrame);

In [8]:
# Replace empty strings with missing

for col in names(d)[4:end]
    # Promote the column to accept missing values
    d[!, col] = convert(Array{Union{Missing, eltype(d[!, col])}}, d[!, col])
    replace!(d[!, col], "" => missing)
end

In [9]:
d

Row,Name,FundId,SecId,Morningstar Category \n1990-01,Morningstar Category \n1990-02,Morningstar Category \n1990-03,Morningstar Category \n1990-04,Morningstar Category \n1990-05,Morningstar Category \n1990-06,Morningstar Category \n1990-07,Morningstar Category \n1990-08,Morningstar Category \n1990-09,Morningstar Category \n1990-10,Morningstar Category \n1990-11,Morningstar Category \n1990-12,Morningstar Category \n1991-01,Morningstar Category \n1991-02,Morningstar Category \n1991-03,Morningstar Category \n1991-04,Morningstar Category \n1991-05,Morningstar Category \n1991-06,Morningstar Category \n1991-07,Morningstar Category \n1991-08,Morningstar Category \n1991-09,Morningstar Category \n1991-10,Morningstar Category \n1991-11,Morningstar Category \n1991-12,Morningstar Category \n1992-01,Morningstar Category \n1992-02,Morningstar Category \n1992-03,Morningstar Category \n1992-04,Morningstar Category \n1992-05,Morningstar Category \n1992-06,Morningstar Category \n1992-07,Morningstar Category \n1992-08,Morningstar Category \n1992-09,Morningstar Category \n1992-10,Morningstar Category \n1992-11,Morningstar Category \n1992-12,Morningstar Category \n1993-01,Morningstar Category \n1993-02,Morningstar Category \n1993-03,Morningstar Category \n1993-04,Morningstar Category \n1993-05,Morningstar Category \n1993-06,Morningstar Category \n1993-07,Morningstar Category \n1993-08,Morningstar Category \n1993-09,Morningstar Category \n1993-10,Morningstar Category \n1993-11,Morningstar Category \n1993-12,Morningstar Category \n1994-01,Morningstar Category \n1994-02,Morningstar Category \n1994-03,Morningstar Category \n1994-04,Morningstar Category \n1994-05,Morningstar Category \n1994-06,Morningstar Category \n1994-07,Morningstar Category \n1994-08,Morningstar Category \n1994-09,Morningstar Category \n1994-10,Morningstar Category \n1994-11,Morningstar Category \n1994-12,Morningstar Category \n1995-01,Morningstar Category \n1995-02,Morningstar Category \n1995-03,Morningstar Category \n1995-04,Morningstar Category \n1995-05,Morningstar Category \n1995-06,Morningstar Category \n1995-07,Morningstar Category \n1995-08,Morningstar Category \n1995-09,Morningstar Category \n1995-10,Morningstar Category \n1995-11,Morningstar Category \n1995-12,Morningstar Category \n1996-01,Morningstar Category \n1996-02,Morningstar Category \n1996-03,Morningstar Category \n1996-04,Morningstar Category \n1996-05,Morningstar Category \n1996-06,Morningstar Category \n1996-07,Morningstar Category \n1996-08,Morningstar Category \n1996-09,Morningstar Category \n1996-10,Morningstar Category \n1996-11,Morningstar Category \n1996-12,Morningstar Category \n1997-01,Morningstar Category \n1997-02,Morningstar Category \n1997-03,Morningstar Category \n1997-04,Morningstar Category \n1997-05,Morningstar Category \n1997-06,Morningstar Category \n1997-07,Morningstar Category \n1997-08,Morningstar Category \n1997-09,Morningstar Category \n1997-10,Morningstar Category \n1997-11,Morningstar Category \n1997-12,Morningstar Category \n1998-01,⋯
,String,String15,String15,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,⋯
1,@rgentum Canadian Equity Portfolio,FS000091C6,F00000NE43,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,mi

I've written a new script to clean the formatting of the raw csv files. Testing the read of those files here:

In [6]:
df = CSV.read("../Data/Refined Data/Mutual Funds/Cleaned Files/monthly-morningstar-category/mf_monthly-morningstar-category_part-1.csv", DataFrame);

In [8]:
first(df,20)

Row,name,fundid,secid,morningstar-category_1990-01,morningstar-category_1990-02,morningstar-category_1990-03,morningstar-category_1990-04,morningstar-category_1990-05,morningstar-category_1990-06,morningstar-category_1990-07,morningstar-category_1990-08,morningstar-category_1990-09,morningstar-category_1990-10,morningstar-category_1990-11,morningstar-category_1990-12,morningstar-category_1991-01,morningstar-category_1991-02,morningstar-category_1991-03,morningstar-category_1991-04,morningstar-category_1991-05,morningstar-category_1991-06,morningstar-category_1991-07,morningstar-category_1991-08,morningstar-category_1991-09,morningstar-category_1991-10,morningstar-category_1991-11,morningstar-category_1991-12,morningstar-category_1992-01,morningstar-category_1992-02,morningstar-category_1992-03,morningstar-category_1992-04,morningstar-category_1992-05,morningstar-category_1992-06,morningstar-category_1992-07,morningstar-category_1992-08,morningstar-category_1992-09,morningstar-category_1992-10,morningstar-category_1992-11,morningstar-category_1992-12,morningstar-category_1993-01,morningstar-category_1993-02,morningstar-category_1993-03,morningstar-category_1993-04,morningstar-category_1993-05,morningstar-category_1993-06,morningstar-category_1993-07,morningstar-category_1993-08,morningstar-category_1993-09,morningstar-category_1993-10,morningstar-category_1993-11,morningstar-category_1993-12,morningstar-category_1994-01,morningstar-category_1994-02,morningstar-category_1994-03,morningstar-category_1994-04,morningstar-category_1994-05,morningstar-category_1994-06,morningstar-category_1994-07,morningstar-category_1994-08,morningstar-category_1994-09,morningstar-category_1994-10,morningstar-category_1994-11,morningstar-category_1994-12,morningstar-category_1995-01,morningstar-category_1995-02,morningstar-category_1995-03,morningstar-category_1995-04,morningstar-category_1995-05,morningstar-category_1995-06,morningstar-category_1995-07,morningstar-category_1995-08,morningstar-category_1995-09,morningstar-category_1995-10,morningstar-category_1995-11,morningstar-category_1995-12,morningstar-category_1996-01,morningstar-category_1996-02,morningstar-category_1996-03,morningstar-category_1996-04,morningstar-category_1996-05,morningstar-category_1996-06,morningstar-category_1996-07,morningstar-category_1996-08,morningstar-category_1996-09,morningstar-category_1996-10,morningstar-category_1996-11,morningstar-category_1996-12,morningstar-category_1997-01,morningstar-category_1997-02,morningstar-category_1997-03,morningstar-category_1997-04,morningstar-category_1997-05,morningstar-category_1997-06,morningstar-category_1997-07,morningstar-category_1997-08,morningstar-category_1997-09,morningstar-category_1997-10,morningstar-category_1997-11,morningstar-category_1997-12,morningstar-category_1998-01,⋯
,String,String15?,String15,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,String?,⋯
1,@rgentum Canadian Equity Portfolio,FS000091C6,F00000NE43,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,missing,mis

It worked!